In [1]:
import os 
%pwd

'c:\\Users\\HP\\Desktop\\MLops\\datascience-project\\research'

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\HP\\Desktop\\MLops\\datascience-project'

In [3]:

import os
from dotenv import load_dotenv

# Load env file
load_dotenv()

os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/TahaShaikhh10/datascience-project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="TahaShaikhh10"
os.environ["MLFLOW_TRACKING_PASSWORD"]="bcd441fefded4edd7b1217aa2d2924161112a8b3"

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [5]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories,save_json

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/TahaShaikhh10/datascience-project.mlflow"


        )
        return model_evaluation_config


In [7]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [18]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
    
    # Saving metrics as local json
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

    # ✅ Safe model logging
            if tracking_url_type_store == "file":
                mlflow.sklearn.log_model(model, "model")
            else:
                # Remote store (like DagsHub) → skip logging
                print("⚠️ Skipping model logging on remote MLflow server (not supported).")



In [19]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-08-25 03:41:52,035: INFO: common: YAML file config\config.yaml loaded successfully.:]
[2025-08-25 03:41:52,038: INFO: common: YAML file params.yaml loaded successfully.:]
[2025-08-25 03:41:52,041: INFO: common: YAML file schema.yaml loaded successfully.:]
[2025-08-25 03:41:52,044: INFO: common: created directory at: artifacts:]
[2025-08-25 03:41:52,045: INFO: common: created directory at: artifacts/model_evaluation:]
[2025-08-25 03:41:52,472: INFO: common: JSON file saved at: artifacts\model_evaluation\metrics.json:]
⚠️ Skipping model logging on remote MLflow server (not supported).
🏃 View run hilarious-midge-943 at: https://dagshub.com/TahaShaikhh10/datascience-project.mlflow/#/experiments/0/runs/27b9bb271edf4af4b6783c94d9c6c6bc
🧪 View experiment at: https://dagshub.com/TahaShaikhh10/datascience-project.mlflow/#/experiments/0
